# Scraping the Data
_Author_: https://github.com/raffysantayana

## Goal
Use the US Securities and Exchange Commision's (SEC) electronic filing system to programmatically parse and organize the data to later be explored, analyzed, and modeled.

## Overview
SEC archives quarterly reports from various filing entities such as Netflix Inc. (NFLX) and American Express Co. (AXP).

WRITE SOMETHING HERE TALKING ABOUT THE API

## Using the SEC API
Note: This requires a subscription of $55/month to make 100+ requests.
```python
import time
import pandas as pd
from sec_api import QueryApi

# main dataframe we will append each query results to
df = pd.DataFrame()

# paste your api key below
sec_api_key:str = 'api_key' # 'a71896086f47a9ae5928bae84adfaff594ec0a1dcbb0bcc3db52ee3aa0f8e15c'
query_api = QueryApi(api_key = sec_api_key)

base_query = {
  "query": "PLACEHOLDER", # this will be set during runtime 
  "from": "0",
  "size": "200", # dont change this
  # sort by filedAt
  "sort": [{ "filedAt": { "order": "desc" } }]
}

# open the file we use to store the filing URLs
log_file = open("filing_urls.txt", "a")

# start with filings filed in 2021, then 2020, 2019, ... up to 2010 
# uncomment line below to fetch all filings filed in 2022-2010
# for year in range(2021, 2009, -1):
for year in range(2024, 1996, -1):
    print("starting {year}".format(year=year))
    # a single search universe is represented as a month of the given year
    
    for month in range(1, 13, 1):
        # get 10-Q and 10-Q/A filings filed in year and month
        # resulting query example: "formType:\"10-Q\" AND filedAt:[2021-01-01 TO 2021-01-31]"
        universe_query = \
            "formType:\"10-Q\" AND " + \
            "filedAt:[{year}-{month:02d}-01 TO {year}-{month:02d}-31]" \
            .format(year=year, month=month)
    
    print(universe_query)
    # set new query universe for year-month combination
    base_query["query"] = universe_query;

    # paginate through results by increasing "from" parameter 
    # until we don't find any matches anymore
    # uncomment line below to fetch 10,000 filings
    for from_batch in range(0, 999_800, 200): 
    # for from_batch in range(0, 400, 200):
        # set new "from" starting position of search 
        base_query["from"] = from_batch;

        # submit request
        response = query_api.get_filings(base_query)
        # building a temp dataframe of the recent query
        temp_df = pd.DataFrame.from_records(response['filings'])
        # concatenating the temp dataframe to the main dataframe
        df = pd.concat([df, temp_df])
        print(f'df.shape = {df.shape}')
        
        # no more filings in search universe
        if len(response["filings"]) == 0:
            break;
            
        # for each filing, only save the URL pointing to the filing itself 
        # and ignore all other data. 
        # the URL is set in the dict key "linkToFilingDetails"
        urls_list = list(map(lambda x: x["linkToFilingDetails"], response["filings"]))
        
        # transform list of URLs into one string by joining all list elements
        # and add a new-line character between each element.
        urls_string = "\n".join(urls_list) + "\n"
      
        log_file.write(urls_string)

log_file.close()
```

## Web Scraping
SEC has an electrtonic filing system Electronic Data Gathering, Analysis, and Retrieval (EDGAR) that started around 1995 to archive reports such as quarterly 10Q. This system has a RESTful API at [this URL](https://www.sec.gov/edgar/sec-api-documentation) to retrieve report information. Each entity’s current filing history is available at the following URL where CIK_number is an entity's 10 digit CIK number: `
https://data.sec.gov/submissions/CIK{CIK_number}.json`

The returning json contains information such as `accessionNumber`, and `primaryDocument` where the index of the `accessionNumber`is associated with the index of `primaryDocument`. Using these two pieces of info together with the CIK number allows us to construct a url to access all filings for that CIK. Our goal is to specifically analyze quarterly reports, so we will filter results based off of `form` value of "10-Q". The URL we will construct will be:
`https://www.sec.gov/Archives/edgar/data/{CIK_number}/{accessionNumber}/{primaryDocument}`

For example, https://www.sec.gov/Archives/edgar/data/0001445815/000149315224015525/form10-qa.htm

A list of all CIK numbers to iterate through can be found [here](https://www.sec.gov/Archives/edgar/cik-lookup-data.txt)

In [1]:
import time
import requests
import pandas as pd

In [2]:
tickers_url:str = r'https://www.sec.gov/files/company_tickers.json'
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(tickers_url, headers=headers)
if response.status_code != 200:
    raise Exception("Failed to get a 200 status code")
else:
    print(f"Successful response from {tickers_url}.")
raw_tickers = response.json()
print(f"{len(raw_tickers)} retrieved.")

Successful response from https://www.sec.gov/files/company_tickers.json.
9610 retrieved.


In [3]:
tickers = pd.DataFrame(columns=['cik_str', 'ticker', 'title'])

for i in range(len(raw_tickers)):
    tickers.loc[f"{i}"] = raw_tickers[f"{i}"]

In [4]:
tickers.shape

(9610, 3)

In [5]:
tickers.dtypes

cik_str     int64
ticker     object
title      object
dtype: object

In [6]:
tickers.head()

,cik_str,ticker,title
0,789019,MSFT,MICROSOFT CORP
1,320193,AAPL,Apple Inc.
2,1045810,NVDA,NVIDIA CORP
3,1652044,GOOGL,Alphabet Inc.
4,1018724,AMZN,AMAZON COM INC


In [7]:
tickers.tail()

,cik_str,ticker,title
9605,1804469,GFAIW,"Guardforce AI Co., Ltd."
9606,1805385,EVLVW,"Evolv Technologies Holdings, Inc."
9607,1816613,MKFG-WT,Markforged Holding Corp
9608,1818331,WGSWW,GeneDx Holdings Corp.
9609,1819074,PTNYD,ParcelPal Logistics Inc.


In [8]:
tickers.to_csv("../data/tickers.csv")

In [9]:
with open('../data/all_submissions_w_duplicates.txt', 'w') as the_file:
    for cik in tickers['cik_str']:
        # Write to all_submissions.txt the url for the given cik number with
        # leading zeroes until 10 digits are reached
        the_file.write(f'https://data.sec.gov/submissions/CIK{cik:010d}.json\n')

In [11]:
with open('../data/all_submissions_w_duplicates.txt', 'r') as dup_file:
    lines = dup_file.readlines()
    with open('../data//all_submissions.txt', 'w') as final_file:
        for unique_line in set(lines):
            final_file.write(unique_line)

## Validating URLs
The cell below will iterate through each URL in `all_submissions.txt` and validate that they each provide a valid response. This only needs to be validated once assuming that SEC does not remove any of these submissions. With this assumption, the below code block will become markdown and will show the results.

```python
urls = open('../data/all_submissions.txt', 'r')
lines = urls.readlines()
counter = 1
for line in lines:
    response = requests.get(line.strip(), headers=headers)
    if response.status_code != 200:
        print(f'{line.strip()} might not be a valid URL. Status code {response.status_code} received. This URL is on line {counter}.')
        counter -= 1
    # print the progress
    print(f'{counter:05d}/{len(lines)}', end='\r')
    time.sleep(5)
    counter += 1
if counter - 1 == len(lines):
    validation_file = open('../data/url_validation.txt', 'w')
    validation_file.write('All URLs in all_submissions.txt have passed validation')
```
Output:
10352/10352

## Filtering URLs of CIK Submissions for 10Q Filings
Now that we have URLs for all CIK numbers that detail all submissions that these entities have provided, we can move on to filter for the specific filing we want to train our model on. For this project, we will focus on 10Q quarterly reports. To show how we will filter for only 10Q

In [12]:
urls = open('../data/all_submissions.txt', 'r')
lines = urls.readlines()

# Get the 0th ticker
response = requests.get(lines[0].strip(), headers=headers)
print(f'Status code {response.status_code} received for URL {lines[0].strip()}')

Status code 200 received for URL https://data.sec.gov/submissions/CIK0000858446.json


In [13]:
raw_json = response.json()

In [14]:
raw_json['tickers']

['IHG']

In [15]:
raw_json['exchanges']

['NYSE']

## Validating the number of values in each column
A row in our dataframe will consist of the below values as well as the accession number above. These values will each be a column in the dataframe and there should be a value - non-null or null/empty string - for each submission for each column.

In [16]:
recent_filings = raw_json['filings']['recent']
recent_filings['accessionNumber']
print(len(recent_filings['accessionNumber']))
print(recent_filings['accessionNumber'][3])

922
0001654954-24-005484


In [17]:
recent_filings['filingDate']
print(len(recent_filings['filingDate']))
print(recent_filings['filingDate'][3])

922
2024-05-03


In [18]:
recent_filings['reportDate']
print(len(recent_filings['reportDate']))
print(recent_filings['reportDate'][3])

922
2024-05-03


In [19]:
recent_filings['acceptanceDateTime']
print(len(recent_filings['acceptanceDateTime']))
print(recent_filings['acceptanceDateTime'][3])

922
2024-05-03T06:29:07.000Z


In [20]:
recent_filings['act']
print(len(recent_filings['act']))
print(recent_filings['act'][3])

922
34


In [21]:
recent_filings['form']
print(len(recent_filings['form']))
print(recent_filings['form'][3])

922
6-K


In [22]:
recent_filings['fileNumber']
print(len(recent_filings['fileNumber']))
print(recent_filings['fileNumber'][3])

922
001-10409


In [23]:
recent_filings['filmNumber']
print(len(recent_filings['filmNumber']))
print(recent_filings['filmNumber'][3])

922
24910706


In [24]:
recent_filings['items']
print(len(recent_filings['items']))
print(recent_filings['items'][3])

922



In [25]:
recent_filings['size']
print(len(recent_filings['size']))
print(recent_filings['size'][3])

922
41029


In [26]:
recent_filings['primaryDocument']
print(len(recent_filings['primaryDocument']))
print(recent_filings['primaryDocument'][3])

922
a0924n.htm


In [27]:
recent_filings['isXBRL']
print(len(recent_filings['isXBRL']))
print(recent_filings['isXBRL'][3])

922
0


In [28]:
recent_filings['isInlineXBRL']
print(len(recent_filings['isInlineXBRL']))
print(recent_filings['isInlineXBRL'][3])

922
0


In [29]:
recent_filings['primaryDocDescription']
print(len(recent_filings['primaryDocDescription']))
print(recent_filings['primaryDocDescription'][3])

922
CHANGES TO SYSTEM FUND ARRANGEMENTS


In [98]:
# Make a function that loops through the length of response.json()['filings']['recent'] and populates a list
# whose items are in the order of the dataframe columns
# start should start at which cik_index to start at (line in all_submissions.txt)
def extract_10qs(cik_url_index=0):
    line_counter = cik_url_index
    tickers = []
    exchanges = []
    accession_numbers = []
    filing_dates = []
    report_dates = []
    acceptance_datetimes = []
    acts = []
    forms = []
    file_numbers = []
    film_numbers = []
    items = []
    sizes = []
    primary_documents = []
    is_XBRLs = []
    is_inline_XBRLs = []
    primary_doc_descriptions = []
    sources = []
    has_multi_tickers = []
    has_multi_exchanges = []
    all_submissions_line_numbers = []
    
    with open('../data/all_submissions.txt', 'r') as file_reader:
        lines = file_reader.readlines()
        line = lines[cik_url_index]
        #for cik_index in range(len(lines)):
        # for line in lines[start:]:
            # Skip cik_index that is less than the specified starting index
            # print(f'cik_index {cik_index} >= start {start} = {cik_index >= start}')
            # if cik_index >= start:
        print(f'Extracting reports from URL {line}', end='\r')
        response = requests.get(line.strip(), headers=headers)

        # WAIT 1 SECOND TO NOT DDOS THE GOVERNMENT
        time.sleep(1)
        if response.status_code != 200:
            print(f'Status code {response.status_code} received for URL {line.strip()}. URL on line {line_counter + 1}', end='\r')
        json = response.json()
        curr_ticker = json['tickers']
        curr_exchange = json['exchanges']
        ticker_filings = json['filings']['recent']
        for i_curr_ticker_filings in range(len(ticker_filings['accessionNumber'])):
            if ticker_filings['form'][i_curr_ticker_filings] == '10-Q':
                if len(curr_ticker) == 0:
                    tickers.append(None)
                else:
                    tickers.append(curr_ticker[0])
                if len(curr_exchange) == 0:
                    exchanges.append(None)
                else:
                    exchanges.append(curr_exchange[0])
                accession_numbers.append(ticker_filings['accessionNumber'][i_curr_ticker_filings])
                filing_dates.append(ticker_filings['filingDate'][i_curr_ticker_filings])
                report_dates.append(ticker_filings['reportDate'][i_curr_ticker_filings])
                acceptance_datetimes.append(ticker_filings['acceptanceDateTime'][i_curr_ticker_filings])
                acts.append(ticker_filings['act'][i_curr_ticker_filings])
                forms.append(ticker_filings['form'][i_curr_ticker_filings])
                file_numbers.append(ticker_filings['fileNumber'][i_curr_ticker_filings])
                film_numbers.append(ticker_filings['filmNumber'][i_curr_ticker_filings])
                items.append(ticker_filings['items'][i_curr_ticker_filings])
                sizes.append(ticker_filings['size'][i_curr_ticker_filings])
                primary_documents.append(ticker_filings['primaryDocument'][i_curr_ticker_filings])
                is_XBRLs.append(ticker_filings['isXBRL'][i_curr_ticker_filings])
                is_inline_XBRLs.append(ticker_filings['isInlineXBRL'][i_curr_ticker_filings])
                primary_doc_descriptions.append(ticker_filings['primaryDocDescription'][i_curr_ticker_filings])
                sources.append(line)
                if len(curr_ticker) > 1:
                    has_multi_tickers.append(1)
                else:
                    has_multi_tickers.append(0)
                if len(curr_exchange) > 1:
                    has_multi_exchanges.append(1)
                else:
                    has_multi_exchanges.append(0)
                all_submissions_line_numbers.append(cik_url_index + 1)
    return pd.DataFrame({
        'ticker': tickers,
        'exchange': exchanges,
        'accession_number': accession_numbers,
        'filing_date': filing_dates,
        'report_date': report_dates,
        'acceptance_datetime': acceptance_datetimes,
        'act': acts,
        'form': forms,
        'file_number': file_numbers,
        'film_number': film_numbers,
        'items': items,
        'size': sizes,
        'primary_document': primary_documents,
        'is_XBRL': is_XBRLs,
        'is_inline_XBRL': is_inline_XBRLs,
        'primary_doc_description': primary_doc_descriptions,
        'source': [source.strip() for source in sources],
        'has_multi_ticker': has_multi_tickers,
        'has_multi_exchange': has_multi_exchanges,
        'all_submissions_line_number': all_submissions_line_numbers
    })

In [99]:
functional_test_df = extract_10qs(cik_url_index=5)
functional_test_df.shape

Extracting reports from URL https://data.sec.gov/submissions/CIK0001145604.json


(9, 20)

In [100]:
functional_test_df.dtypes

ticker                         object
exchange                       object
accession_number               object
filing_date                    object
report_date                    object
acceptance_datetime            object
act                            object
form                           object
file_number                    object
film_number                    object
items                          object
size                            int64
primary_document               object
is_XBRL                         int64
is_inline_XBRL                  int64
primary_doc_description        object
source                         object
has_multi_ticker                int64
has_multi_exchange              int64
all_submissions_line_number     int64
dtype: object

The `extract_10qs()` function can now be used to extract a maximum of `extraction_count` number of reports. It will return a dataframe of all 10q reports between the range of `start` and (`start` + `extraction_count`). The resulting dataframe can then be concatenated to the original dataframe. Gathering this metadata will take some time, so it is a huge benefit to periodically pause extraction, concatenate the incremental progress to the original dataframe, and then save the dataframe to a csv to continue progress at a later time.

In [101]:
functional_test_df.index.max()

8

In [102]:
df = pd.DataFrame(columns=['ticker', 'exchange', 'accession_number', 'filing_date', 'report_date', 'acceptance_datetime', 'act',
                              'form', 'file_number', 'film_number', 'items', 'size', 'primary_document', 'is_XBRL', 'is_inline_XBRL', 
                              'primary_doc_description', 'source', 'has_multi_ticker', 'has_multi_exchange', 'all_submissions_line_number'])

In [107]:
with open('../data/all_submissions.txt', 'r') as submissions:
    lines = submissions.readlines()
    for i_line in range(len(lines)):
        df = pd.concat([df, extract_10qs(i_line)], axis=0)
        df.to_csv('../data/debug/all_10qs.csv')

Extracting reports from URL https://data.sec.gov/submissions/CIK0000858446.json
Extracting reports from URL https://data.sec.gov/submissions/CIK0001161814.json
Extracting reports from URL https://data.sec.gov/submissions/CIK0000840715.json
Extracting reports from URL https://data.sec.gov/submissions/CIK0001004434.json
Extracting reports from URL https://data.sec.gov/submissions/CIK0000801961.json
Extracting reports from URL https://data.sec.gov/submissions/CIK0001145604.json
Extracting reports from URL https://data.sec.gov/submissions/CIK0001565687.json
Extracting reports from URL https://data.sec.gov/submissions/CIK0001281761.json
Extracting reports from URL https://data.sec.gov/submissions/CIK0001633978.json
Extracting reports from URL https://data.sec.gov/submissions/CIK0000109177.json
Extracting reports from URL https://data.sec.gov/submissions/CIK0001861063.json
Extracting reports from URL https://data.sec.gov/submissions/CIK0000004457.json
Extracting reports from URL https://data

In [111]:
fh

all_submissions_line_number
12      182
3       168
4       129
3389     92
2604     92
       ... 
2094      1
1272      1
1285      1
3637      1
519       1
Name: count, Length: 5870, dtype: int64